In [ ]:
# packages are loaded
import xarray as xr
import pint_xarray
import glob
import netCDF4 as nc4
import os
import pandas as pd
from   easymore import Easymore
import numpy as np
import geopandas   as  gpd

In [28]:
# inputs
# Set the folder path where the remapped .nc file is located for MESH (it can be any remapped nc file)
path_to_save = '/scratch/mia725/SCRB/SUMMA'
gistool_output = '/scratch/mia725/SCRB/gistool-outputs'
subbasins_shapefile = '/scratch/mia725/SCRB/MERIT_geofabric/extracted_subbasins.shp'
rivers_shapefile = '/scratch/mia725/SCRB/MERIT_geofabric/extracted_rivers.shp'

In [29]:
forcing = xr.open_dataset(os.path.join(path_to_save,'SUMMA_forcing.nc'))
forcing

<xarray.Dataset>
Dimensions:    (time: 8760, hru: 13)
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01T13:00:00 ... 1980-12-31T12:00:00
    hruId      (hru) int32 ...
  * hru        (hru) int32 71030562 71030476 71030488 ... 71034539 71034538
Data variables:
    airpres    (time, hru) float64 ...
    spechum    (time, hru) float64 ...
    airtemp    (time, hru) float64 ...
    windspd    (time, hru) float64 ...
    pptrate    (time, hru) float64 ...
    SWRadAtm   (time, hru) float64 ...
    LWRadAtm   (time, hru) float64 ...
    latitude   (hru) float64 ...
    longitude  (hru) float64 ...
    crs        int32 ...
    data_step  int64 ...
Attributes:
    author:       University of Calgary
    license:      GNU General Public License v3 (or any later version)
    purpose:      Create forcing .nc file for MESH
    Conventions:  CF-1.6
    history:      Created using MESHFlow version v0.1.0.dev1

In [48]:
# prepare data by merging the spatial fields into one dataframe
# read merit geofabric
# read rivers
riv = gpd.read_file(rivers_shapefile)
# reorder river to follow the same order as the hru in the forcing file
riv = riv.set_index('COMID').loc[forcing['hru']].reset_index()

# read catchments
cat = gpd.read_file(subbasins_shapefile)
# reorder river to follow the same order as the hru in the forcing file
cat = cat.set_index('COMID').loc[forcing['hru']].reset_index()

,COMID,lengthkm,lengthdir,sinuosity,slope,uparea,order,strmDrop_t,slope_taud,NextDownID,maxup,up1,up2,up3,up4,unitarea
0,71030562,12.605595,7.319903,1.722099,0.000277,52.894817,1,3.5,0.000277,71029990,0,0,0,0,0,52.894817
1,71030476,1.572101,1.017370,1.545260,0.001461,34.295081,1,2.3,0.001461,71029990,0,0,0,0,0,34.295081
2,71030488,11.714294,6.815784,1.718701,0.000392,51.716595,1,4.6,0.000392,71029926,0,0,0,0,0,51.716595
3,71029990,1.462706,1.126778,1.298132,0.000273,87.829467,2,0.4,0.000273,71029926,2,71030476,71030562,0,0,0.639570
4,71030449,3.717695,2.582753,1.439431,0.001101,36.241250,1,4.1,0.001101,71034853,0,0,0,0,0,36.241250
5,71029926,4.971691,3.553336,1.399161,0.000623,143.969752,2,3.1,0.000623,71034853,2,71029990,71030488,0,0,4.423690
6,71039254,11.720198,6.016223,1.948099,0.000392,71.346456,1,4.6,0.000392,71034609,0,0,0,0,0,71.346456
7,71034853,12.455554,7.555539,1.648533,0.000369,228.319337,2,4.6,0.000369,71034609,2,71029926,71030449,0,0,48.108335
8,71039052,12.462729,7.294305,1.708556,0.001041,54.875958,1,13.0,0.001041,71034539,0,0,0,0,0,54.875958
9,71034609,16.763706,10.435159,1.606464,0.000739,349.539910,2,12.4,0.000739,71034539,2,71034853,71039254,0,0,49.874117


In [50]:
# read elevation stats
elev_stats = pd.read_csv(os.path.join(gistool_output, 'modified_domain_stats_elv.csv'))
# reorder df to follow the same order as the hru in the forcing file
elev_stats = elev_stats.set_index('COMID').loc[forcing['hru']].reset_index()
#rename columns except COMID
elev_stats = elev_stats.rename(columns=lambda x: x + '_elev' if x != 'COMID' else x)

,COMID,min_elev,max_elev,mean_elev,median_elev
0,71030562,528.500000,542.600037,534.345093,534.332413
1,71030476,528.500000,541.500000,535.878235,536.053374
2,71030488,528.100037,543.400024,535.666443,536.317408
3,71029990,528.100037,531.100037,529.174561,529.172650
4,71030449,526.600037,541.000000,534.652222,535.026395
5,71029926,525.000000,533.799988,528.749268,528.876652
6,71039254,520.900024,543.200012,531.857849,531.300621
7,71034853,520.400024,538.299988,529.174194,528.683972
8,71039052,508.000000,539.200012,523.450134,523.248459
9,71034609,508.800018,533.900024,521.130737,520.858053


In [ ]:
# read soil and landcover data
soil_stats = pd.read_csv(os.path.join(gistool_output, 'modified_domain_stats_soil_classes.csv'))
# reorder df to follow the same order as the hru in the forcing file
soil_stats = soil_stats.set_index('COMID').loc[forcing['hru']].reset_index()

landuse_stats = pd.read_csv(os.path.join(gistool_output, 'modified_domain_stats_NA_NALCMS_landcover_2020_30m.csv'))
# reorder df to follow the same order as the hru in the forcing file
landuse_stats = landuse_stats.set_index('COMID').loc[forcing['hru']].reset_index()
soil_landuse_stats = landuse_stats.merge(soil_stats, on='COMID')

In [ ]:
# link NALCMS to USGS ids in the VEGPARM.TBL
# Manually matched NALCMS to USGS land cover types
matched_landuse = [
    (0, 'Unknown', None, None),
    (1, 'Temperate or sub-polar needleleaf forest', 14, 'Evergreen Needleleaf Forest'),
    (2, 'Sub-polar taiga needleleaf forest', 12, 'Deciduous Needleleaf Forest'),
    (3, 'Tropical or sub-tropical broadleaf evergreen forest', 13, 'Evergreen Broadleaf Forest'),
    (4, 'Tropical or sub-tropical broadleaf deciduous forest', 11, 'Deciduous Broadleaf Forest'),
    (5, 'Temperate or sub-polar broadleaf deciduous forest', 11, 'Deciduous Broadleaf Forest'),
    (6, 'Mixed forest', 15, 'Mixed Forest'),
    (7, 'Tropical or sub-tropical shrubland', 8, 'Shrubland'),
    (8, 'Temperate or sub-polar shrubland', 8, 'Shrubland'),
    (9, 'Tropical or sub-tropical grassland', 10, 'Savanna'),
    (10, 'Temperate or sub-polar grassland', 7, 'Grassland'),
    (11, 'Sub-polar or polar shrubland-lichen-moss', 8, 'Shrubland'),
    (12, 'Sub-polar or polar grassland-lichen-moss', 7, 'Grassland'),
    (13, 'Sub-polar or polar barren-lichen-moss', 19, 'Barren or Sparsely Vegetated'),
    (14, 'Wetland', 17, 'Herbaceous Wetland'),
    (15, 'Cropland', 2, 'Dryland Cropland and Pasture'),
    (16, 'Barren lands', 19, 'Barren or Sparsely Vegetated'),
    (17, 'Urban', 1, 'Urban and Built-Up Land'),
    (18, 'Water', 16, 'Water Bodies'),
    (19, 'Snow and Ice', 24, 'Snow or Ice')
]

# Create DataFrame
matched_lanuse = pd.DataFrame(matched_landuse, columns=['NALCMS_ID', 'NALCMS_Description', 'USGS_ID', 'USGS_Description'])

# Create a dictionary for mapping NALCMS IDs to USGS IDs
nalcms_to_usgs = dict(zip(matched_lanuse['NALCMS_ID'], matched_lanuse['USGS_ID']))

In [ ]:
# link NALCMS to USGS ids in the VEGPARM.TBL
# Manually matched soilgrid to ROSETTA types
matched_soils = [
    (1, 'clay', 1, 'CLAY'),
    (2, 'silty clay', 10, 'SILTY CLAY'),
    (3, 'sandy clay', 6, 'SANDY CLAY'),
    (4, 'clay loam', 2, 'CLAY LOAM'),
    (5, 'silty clay loam', 11, 'SILTY CLAY LOAM'),
    (6, 'sandy clay loam', 7, 'SANDY CLAY LOAM'),
    (7, 'loam', 3, 'LOAM'),
    (8, 'silty loam', 12, 'SILT LOAM'),
    (9, 'sandy loam', 8, 'SANDY LOAM'),
    (10, 'silt', 9, 'SILT'),
    (11, 'loamy sand', 4, 'LOAMY SAND'),
    (12, 'sand', 5, 'SAND')
]

# Create DataFrame
matched_soils = pd.DataFrame(matched_soils, columns=['Soilgrid_ID', 'Soilgrid_Description', 'ROSETTA_ID', 'ROSETTA_Description'])
# Create a dictionary for mapping Soilgrid IDs to ROSETTA IDs
soilgrid_to_rosetta = dict(zip(matched_soils['Soilgrid ID'], matched_soils['ROSETTA ID']))

In [ ]:
# replace 

In [52]:
# merge riv, cat, and elev_stat in one dataframe on COMID
geofabric = riv.merge(cat, on='COMID')
geofabric = geofabric.merge(elev_stats, on='COMID')
geofabric = geofabric.drop(columns=['geometry_x', 'hillslope_x', 'hillslope_y', 'geometry_y'])

,COMID,lengthkm,lengthdir,sinuosity,slope,uparea,order,strmDrop_t,slope_taud,NextDownID,maxup,up1,up2,up3,up4,unitarea,min_elev,max_elev,mean_elev,median_elev
0,71030562,12.605595,7.319903,1.722099,0.000277,52.894817,1,3.5,0.000277,71029990,0,0,0,0,0,52.894817,528.500000,542.600037,534.345093,534.332413
1,71030476,1.572101,1.017370,1.545260,0.001461,34.295081,1,2.3,0.001461,71029990,0,0,0,0,0,34.295081,528.500000,541.500000,535.878235,536.053374
2,71030488,11.714294,6.815784,1.718701,0.000392,51.716595,1,4.6,0.000392,71029926,0,0,0,0,0,51.716595,528.100037,543.400024,535.666443,536.317408
3,71029990,1.462706,1.126778,1.298132,0.000273,87.829467,2,0.4,0.000273,71029926,2,71030476,71030562,0,0,0.639570,528.100037,531.100037,529.174561,529.172650
4,71030449,3.717695,2.582753,1.439431,0.001101,36.241250,1,4.1,0.001101,71034853,0,0,0,0,0,36.241250,526.600037,541.000000,534.652222,535.026395
5,71029926,4.971691,3.553336,1.399161,0.000623,143.969752,2,3.1,0.000623,71034853,2,71029990,71030488,0,0,4.423690,525.000000,533.799988,528.749268,528.876652
6,71039254,11.720198,6.016223,1.948099,0.000392,71.346456,1,4.6,0.000392,71034609,0,0,0,0,0,71.346456,520.900024,543.200012,531.857849,531.300621
7,71034853,12.455554,7.555539,1.648533,0.000369,228.319337,2,4.6,0.000369,71034609,2,71029926,71030449,0,0,48.108335,520.400024,538.299988,529.174194,528.683972
8,71039052,12.462729,7.294305,1.708556,0.001041,54.875958,1,13.0,0.001041,71034539,0,0,0,0,0,54.875958,508.000000,539.200012,523.450134,523.248459
9,71034609,16.763706,10.435159,1.606464,0.000739,349.539910,2,12.4,0.000739,71034539,2,71034853,71039254,0,0,49.874117,508.800018,533.900024,521.130737,520.858053


In [ ]:
# Create a new xarray dataset
attr = xr.Dataset()

# prepare for the SUMMA attr file
attr ['hruId']          = xr.DataArray(geofabric['COMID'].values, dims=('hru'), 
                                       attrs={'long_name': 'Index of hydrological response units (HRU)', 'units': '-'})

attr ['gruId']          = xr.DataArray(geofabric['COMID'].values, dims=('gru'),
                                      attrs={'long_name': 'Index of group of response unit (GRU)', 'units': '-'})

attr ['hru2gruId']      = xr.DataArray(geofabric['COMID'].values, dims=('hru'),
                                      attrs={'long_name': 'Index of GRU to which the HRU belongs', 'units': '-'})

attr ['downHRUindex']   = xr.DataArray(np.zeros(len(geofabric['COMID'])), dims=('hru'),
                                      attrs={'long_name': 'Index of downslope HRU (0 = basin outlet)', 'units': '-'})

attr ['elevation']      = xr.DataArray(geofabric['mean_elev'].values, dims=('hru'),
                                      attrs={'long_name': 'Elevation of HRU\'s centriod point', 'units': 'm'})

attr ['HRUarea']        = xr.DataArray(geofabric['unitarea'].values, dims=('hru'),
                                      attrs={'long_name': 'Area of each HRU', 'units': 'm^2'})

attr ['tan_slope']      = xr.DataArray(geofabric['slope'].values, dims=('hru'),
                                      attrs={'long_name': 'Average tangent slope of HRU', 'units': 'm/m'})

attr ['contourLength']  = xr.DataArray(geofabric['lengthkm'].values*1000, dims=('hru'),
                                        attrs={'long_name': 'ContourLength of HRU', 'units': 'm'})

attr ['slopeTypeIndex'] = xr.DataArray(np.ones(len(geofabric['COMID'])), dims=('hru'),
                                       attrs={'long_name': 'Index defining slope', 'units': '-'})

attr ['soilTypeIndex']  = xr.DataArray(list(map(soilgrid_to_rosetta.get, geofabric['soil_majority'].values)).astype(int), dims=('hru'),
                                        attrs={'long_name': 'Index defining soil type - ROSETTA', 'units': '-'})

attr ['vegTypeIndex']   = xr.DataArray(list(map(nalcms_to_usgs.get, geofabric['LULC_majority'].values)).astype(int), dims=('hru'),
                                       attrs={'long_name': 'Index defining vegetation type - USGS', 'units': '-'})

attr ['mHeight']        = xr.DataArray(np.ones(len(geofabric['COMID']))*40, dims=('hru'),
                                      attrs={'long_name': 'Measurement height above bare ground', 'units': 'm'})


attr

if os.path.isfile(path_to_save+'SUMMA_attributes.nc'):
    os.remove(path_to_save+'SUMMA_attributes.nc')

attr.to_netcdf(path_to_save+'SUMMA_attributes.nc')

In [15]:
# Define dimensions
hru_size = len(attr['hruId'].values)
midSoil_size = 8
midToto_size = 8
ifcToto_size = 9
scalarv_size = 1

# Create a new xarray dataset
ds = xr.Dataset()

# Add dimensions to the dataset
ds['hru'] = xr.DataArray(attr['hruId'].values, dims=('hru'), attrs={'units': '-'})
ds['midSoil'] = xr.DataArray(range(midSoil_size), dims=('midSoil'))
ds['midToto'] = xr.DataArray(range(midToto_size), dims=('midToto'))
ds['ifcToto'] = xr.DataArray(range(ifcToto_size), dims=('ifcToto'))
ds['scalarv'] = xr.DataArray(range(scalarv_size), dims=('scalarv'))

# Add variables to the dataset
ds['hruId'] = xr.DataArray(attr['hruId'].values, dims=('hru'), attrs={'units': '-', 'long_name': 'Index of hydrological response unit (HRU)'})
ds['dt_init'] = xr.DataArray([[3600.0] * hru_size], dims=('scalarv', 'hru'))
ds['nSoil'] = xr.DataArray([[8] * hru_size], dims=('scalarv', 'hru'))
ds['nSnow'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanopyIce'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanopyLiq'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSnowDepth'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSWE'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSfcMeltPond'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarAquiferStorage'] = xr.DataArray([[1.0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarSnowAlbedo'] = xr.DataArray([[0] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanairTemp'] = xr.DataArray([[283.16] * hru_size], dims=('scalarv', 'hru'))
ds['scalarCanopyTemp'] = xr.DataArray([[283.16] * hru_size], dims=('scalarv', 'hru'))
ds['mLayerTemp'] = xr.DataArray([[283.16] * hru_size ] * midToto_size , dims=('midToto', 'hru'))
ds['mLayerVolFracIce'] = xr.DataArray([[0] * hru_size] * midToto_size, dims=('midToto', 'hru'))
ds['mLayerVolFracLiq'] = xr.DataArray([[0.2] * hru_size] * midToto_size, dims=('midToto', 'hru'))
ds['mLayerMatricHead'] = xr.DataArray([[-1] * hru_size] * midToto_size, dims=('midSoil', 'hru'))
# ds['iLayerHeight'] = xr.DataArray([[0.000,0.025,0.100,0.250,0.500,1.000,1.500,2.500,4.000]] * hru_size , dims=( 'hru', 'ifcToto',))
# ds['mLayerDepth'] = xr.DataArray([[0.025,0.075,0.150,0.250,0.500,0.500,1.000,1.500]] * hru_size, dims=( 'hru', 'midToto',))
ds['iLayerHeight'] = xr.DataArray(np.transpose([[0.000,0.025,0.100,0.250,0.500,1.000,1.500,2.500,4.000]] * hru_size) , dims=('ifcToto', 'hru'))
ds['mLayerDepth'] = xr.DataArray(np.transpose([[0.025,0.075,0.150,0.250,0.500,0.500,1.000,1.500]] * hru_size), dims=('midToto',  'hru'))


if os.path.isfile(path_to_save+'SUMMA_coldState.nc'):
    os.remove(path_to_save+'SUMMA_coldState.nc')

ds.to_netcdf(path_to_save+'SUMMA_coldState.nc')



In [16]:
# Define dimensions
hru_size = len(attr['hruId'].values)

# Create a new xarray dataset
ds = xr.Dataset()

# Add dimensions to the dataset
ds['hru'] = xr.DataArray(attr['hruId'].values, dims=('hru'), attrs={'units': '-'})

# Add variables to the dataset
ds['hruId'] = xr.DataArray(attr['hruId'].values, dims=('hru'), attrs={'units': '-', 'long_name': 'Index of hydrological response unit (HRU)'})

if os.path.isfile(path_to_save+'SUMMA_trialParams.nc'):
    os.remove(path_to_save+'SUMMA_trialParams.nc')

ds.to_netcdf(path_to_save+'SUMMA_trialParams.nc')